In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, jsonify
import time

In [53]:
#setup splinter
executable_path= {'executable_path': ChromeDriverManager().install()}
browser= Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/juliankauffmann/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [28]:
#opens NYTimes
browser.visit("https://nytimes.com")

In [74]:
browser.fill('q', 'transgender')

ElementDoesNotExist: no elements could be found with name "q"

In [82]:
browser.find_by_tag('input').fill("hello")

In [73]:
#finds the search icon and clicks it
buttons = browser.find_by_tag('button')

for button in buttons:
    if (button.value=="SEARCH"):
        button.click()
        print('found')
        break




hit


In [ ]:
#fills in the search bar
search_text="transgender"
browser.find_by_tag('input').fill(search_text)

In [ ]:
#finds the 'Go' button and clicks it to start search
buttons = browser.find_by_tag('button')

#the 'Go' button is the +1 after the search icon
found = False 

for button in buttons:
    if (found):
        button.click()
        brea
    if (button.value=="SEARCH"):
        found = True

In [72]:
while True:
    if browser.is_text_present('Julia'):
        print ("Yes, found it! :)")
        break
    else:
        button_next=browser.find_by_id('pnnext').click()

Yes, found it! :)


In [16]:
#function to have beautifulSoup parse url
def parser (theurl):
    
    #pull up url
    browser.visit(theurl)
    html = browser.html
    thesoup = BeautifulSoup(html, 'html.parser')

    return thesoup

In [59]:
parser("https://nytimes.com")

<html lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/" class="nytapp-vi-homepage" data-rh="lang"><head>
    <meta charset="utf-8">
    <title>The New York Times - Breaking News, US News, World News and Videos</title>
    <meta data-rh="true" name="robots" content="noarchive,noodp,noydir"><meta data-rh="true" name="description" content="Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more."><meta data-rh="true" property="og:url" content="https://www.nytimes.com"><meta data-rh="true" property="og:type" content="website"><meta data-rh="true" property="og:title" content="The New York Times - Breaking News, US News, World News and Videos"><meta data-rh="true" property="og:description" content="Live news, investigations, opinion, photos and video by the journalists of T

<html class="nytapp-vi-homepage" data-rh="lang" lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/"><head>
<meta charset="utf-8"/>
<title>The New York Times - Breaking News, US News, World News and Videos</title>
<meta content="noarchive,noodp,noydir" data-rh="true" name="robots"/><meta content="Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news, politics, business, technology, science, health, arts, sports and more." data-rh="true" name="description"/><meta content="https://www.nytimes.com" data-rh="true" property="og:url"/><meta content="website" data-rh="true" property="og:type"/><meta content="The New York Times - Breaking News, US News, World News and Videos" data-rh="true" property="og:title"/><meta content="Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries 

In [4]:
#1)Scrape the Mars News Site and collect the latest News Title and Paragraph Text
url='https://redplanetscience.com/'
soup=parser(url)

#declare variables -- originally I read the above as finding all the latest News Titles and decided to keep it for practice
mars_titles=[]
mars_paragraphs=[]

#scrape titles
results=soup.find_all('div', class_='content_title')

#append titles to list
for result in results:
    mars_titles.append(result.text.strip())

#assign the most recent title to a variable 
news_title=mars_titles[0]

#print results to verify
news_title

"NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet"

In [5]:
#scrape paragraph
results=soup.find_all('div', 'article_teaser_body')

#append paragraphs to list
for result in results:
    mars_paragraphs.append(result.text.strip())
    
#save 1st paragraph
news_p=mars_paragraphs[0]

print (news_p)

The Mars rover has drilled three samples of rock in this clay-enriched region since arriving in July.


In [6]:
#2)Visit the url for the Featured Space Image site here.

#parse feature image site
url='https://spaceimages-mars.com/'
soup=parser(url)

In [7]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image
browser.links.find_by_partial_text('FULL IMAGE').click()

#store new html
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#save feature image source
result=soup.find('img', class_='fancybox-image')
featured_image_url=url+result['src']

print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [8]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table

url='https://galaxyfacts-mars.com/'
tables=pd.read_html(url)

In [9]:
#Use Pandas to convert the data to a HTML table string.
the_table=tables[0].set_index(0)
the_table.rename(columns={1:'Mars',2:'Earth'},inplace=True)
the_table.index.names = ['Description']

#set table to html
the_table_html=the_table.to_html()

#print to verify
the_table

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
#function to pull the image url
def imagePull(hemisphere):
    
    #click on link for hemisphere
    browser.links.find_by_partial_text(hemisphere).click()
    
    #parse the new browser
    html = browser.html
    temp = BeautifulSoup(html, 'html.parser')
    
    #find the large image file
    the_image=temp.find('a',text='Sample')
    
    #move back to original page
    browser.links.find_by_partial_text('Back').click()

    return the_image['href']

#----------------------------------------------------
#list to store dictionary of image url and title
hemisphere_image_urls=[]

#open the page
url='https://marshemispheres.com/'
soup=parser(url)

#finds link names
links=soup.find_all('h3')

#loops through the first four links
for link in links:
    if link.text !='Back':
        the_image_url=f'{url}{imagePull(link.text)}'
        
        #append the list with a dictionary
        hemisphere_image_urls.append({"title": link.text, "img_url":the_image_url})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [11]:
#return a dictionary of all the scraped data
scraped_data={
    "latest_title":news_title,"latest_p":news_p,
    "featured_image":featured_image_url,
    "the_table":the_table_html,
    "the_hemispheres":hemisphere_image_urls    
}

In [103]:
browser.quit()